In [1]:
!pip install lightfm
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.9 MB/s eta 0:00:0000:0100:02
  Preparing metadata (setup.py) ... done
  Using cached py4j-0.10.9.7-py2.py3-none-any.whl.metadata (1.5 kB)
Using cached py4j-0.10.9.7-py2.py3-none-any.whl (200 kB)
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=2423b5c99addec4d303835aa3885298415038db9c6ad5144b520f07919c54955
  Stored in directory: /Users/yaoyi/Library/Caches/pip/wheels/92/09/11/aa01d01a7f005fda8a66ad71d2be7f8aa341bddafb27eee3c7
Successfully built pyspark


In [2]:
import numpy as np
import pandas as pd
from lightfm import LightFM
from lightfm.data import Dataset
from lightfm.evaluation import precision_at_k, auc_score, recall_at_k
from lightfm.cross_validation import random_train_test_split
import ast
import pickle
from collections import deque

/Users/yaoyi/opt/anaconda3/envs/airflow_env/lib/python3.9/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


In [3]:
df = pd.read_csv('ET_steam_clean.csv')
# Assuming `df` is your DataFrame with the structure provided.
df_copy = df

In [3]:
from recommenders.evaluation.python_evaluation import precision_at_k, recall_at_k, diversity, map_at_k, ndcg_at_k, auc
from recommenders.utils.timer import Timer
from recommenders.evaluation.python_evaluation import (rmse, mae, rsquared, exp_var, map_at_k, ndcg_at_k, precision_at_k,
                                                     recall_at_k, get_top_k_items,
                                                     catalog_coverage, distributional_coverage, novelty, diversity, serendipity)
from recommenders.utils.constants import SEED as DEFAULT_SEED
import pyspark
import pyspark.sql.functions as F
from pyspark.sql.window import Window
from pyspark.sql.types import FloatType, IntegerType, LongType, StructType, StructField
from pyspark.ml.feature import Tokenizer, StopWordsRemover
from pyspark.ml.feature import HashingTF, CountVectorizer, VectorAssembler
from pyspark.ml.recommendation import ALS
from recommenders.datasets.spark_splitters import spark_random_split
from recommenders.datasets.python_splitters import python_chrono_split, python_stratified_split
from recommenders.evaluation.spark_evaluation import SparkRankingEvaluation, SparkDiversityEvaluation
from recommenders.utils.spark_utils import start_or_get_spark

In [5]:
df.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'recommendationid', 'author', 'review',
       'timestamp_created', 'timestamp_updated', 'voted_up', 'votes_up',
       'weighted_vote_score', 'app_id', 'title', 'date_release', 'rating',
       'positive_ratio', 'user_reviews', 'price_final', 'price_original', 'os',
       'time_between_comment_and_game', 'days_between_comment_and_game',
       'rating_numerical', 'rating_simplified'],
      dtype='object')

In [11]:
df_copy = df[['author', 'app_id', 'rating_numerical']]
df_copy = df_copy.rename(columns={'author':'userID', 'app_id':'itemID', 'rating_numerical':'rating'})
df_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 417129 entries, 0 to 417128
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype
---  ------  --------------   -----
 0   userID  417129 non-null  int64
 1   itemID  417129 non-null  int64
 2   rating  417129 non-null  int64
dtypes: int64(3)
memory usage: 9.5 MB


In [12]:
train, test = python_stratified_split(
    df_copy, ratio=0.8, seed=42
)

In [13]:

# Filtering out users and items in the test set that do not appear in the training set.
# This is done so that we can see if our model has learnt user's previous item interactions and can recommend relevant items.
test = test[test["userID"].isin(train["userID"].unique())]
test = test[test["itemID"].isin(train["itemID"].unique())]

# Creating a test set which only contains the last interaction for each user. Remaining data of the user is used in the train set
leave_one_out_test = test.groupby("userID").last().reset_index()

test.head()


,userID,itemID,rating
358973,76561197960269213,555260,0
367218,76561197960269409,335920,3
334063,76561197960270054,1816890,2
199570,76561197960271994,1166780,0
289997,76561197960272599,447960,0


In [4]:
import sys
import os
import shutil

import pandas as pd
import numpy as np

from recommenders.utils.timer import Timer
from recommenders.datasets.python_splitters import python_chrono_split, python_stratified_split

from recommenders.models.ncf.dataset import Dataset as NCFDataset

# Importing the NCF model class from the recommenders library
from recommenders.models.ncf.ncf_singlenode import NCF

# importing the evaluation metrics
from recommenders.evaluation.python_evaluation import (rmse, mae, rsquared, exp_var, map_at_k, ndcg_at_k, precision_at_k,
                                                     recall_at_k, get_top_k_items,
                                                     catalog_coverage, distributional_coverage, novelty, diversity, serendipity)
from recommenders.utils.constants import SEED as DEFAULT_SEED

import warnings

# Suppress all warnings
warnings.filterwarnings("ignore")

2024-04-23 16:34:32.998615: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
# top k items to recommend
TOP_K = 10

# Model parameters
# Number of iterations during the training process
EPOCHS = 5
# Batch size means how many user-item pairs you want to predict at once
BATCH_SIZE = 256

# Setting seed to remove any stochasticity and reproduce results
SEED = DEFAULT_SEED  # Set None for non-deterministic results

In [6]:
# Writing the data into csv files
train_file = "train.csv"
test_file = "test.csv"
leave_one_out_test_file = "leave_one_out_test.csv"

# train.to_csv(train_file, index=False)
# test.to_csv(test_file, index=False)
# leave_one_out_test.to_csv(leave_one_out_test_file, index=False)

data = NCFDataset(train_file=train_file, test_file=leave_one_out_test_file, seed=SEED, overwrite_test_file_full=True)

INFO:recommenders.models.ncf.dataset:Indexing train.csv ...
INFO:recommenders.models.ncf.dataset:Indexing leave_one_out_test.csv ...
INFO:recommenders.models.ncf.dataset:Creating full leave-one-out test file leave_one_out_test_full.csv ...
100%|██████████| 19894/19894 [06:46<00:00, 48.99it/s]
INFO:recommenders.models.ncf.dataset:Indexing leave_one_out_test_full.csv ...


In [9]:
model = NCF (
    n_users=data.n_users,
    n_items=data.n_items,
    model_type="NeuMF",
    n_factors=3,
    layer_sizes=[64,24,8],
    n_epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    learning_rate=1e-2,
    verbose=10,
    seed=SEED
)
from tqdm import tqdm
with Timer() as train_time:
    model.fit(data)


print("Took {} seconds for training.".format(train_time.interval))


In [ ]:
# save the model
# https://github.com/recommenders-team/recommenders/issues/1735
dir_path = 'NCF_model_trained'
# # and then while loading depending on the type of your model in this case neumf pass it that dir parameter
# model.load(neumf_dir='dir_path')

In [ ]:
model = NCF(
    n_users=data.n_users,
    n_items=data.n_items,
    model_type="NeuMF",
    n_factors=4,
    layer_sizes=[64,32,16,8],
    n_epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    learning_rate=1e-3,
    verbose=10,
    seed=SEED
)

dir_path = 'NCF_model_trained'
model.load(neumf_dir = dir_path)

model.user2id = data.user2id
model.item2id = data.item2id
model.id2user = data.id2user
model.id2item = data.id2item